## <font color='green'> Least Squares Estimation using Matrix Algebra and Numerical Optimization

* Lecture note 2: p.19

In [53]:
import os
os.chdir('')

In [54]:
import numpy as np 
import pandas as pd 
import math 

raw0 = pd.read_csv('College.csv')
raw0['Private']=pd.get_dummies(raw0['Private'],drop_first=True)
raw0.head()

,Unnamed: 0,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
0,Abilene Christian University,1,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
1,Adelphi University,1,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
2,Adrian College,1,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
3,Agnes Scott College,1,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
4,Alaska Pacific University,1,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15


### <font color='green'> 1) Least Squares Estimation using Matrix Algebra

https://web.stanford.edu/~mrosenfe/soc_meth_proj3/matrix_OLS_NYU_notes.pdf

In [55]:
# convert the dataframe to a numpy array (excluding the first column-college names)
# Exclude column 0 go to the end
raw00 = raw0.iloc[:,1:].values 

* We can suppress scientific notation using "np.set_printoptions" (e.g. np.set_printoptions(precision=2, suppress=True))
* Scientific notation: https://en.wikipedia.org/wiki/Scientific_notation
* np.set_printoptions: https://numpy.org/doc/stable/reference/generated/numpy.set_printoptions.html

In [56]:
raw00

array([[    1.,  1660.,  1232., ...,    12.,  7041.,    60.],
       [    1.,  2186.,  1924., ...,    16., 10527.,    56.],
       [    1.,  1428.,  1097., ...,    30.,  8735.,    54.],
       ...,
       [    1.,  2097.,  1915., ...,    20.,  8323.,    49.],
       [    1., 10705.,  2453., ...,    49., 40386.,    99.],
       [    1.,  2989.,  1855., ...,    28.,  4509.,    99.]])

In [57]:
# Number will show up to second decimal place
np.set_printoptions(precision=2, suppress=True)

In [58]:
raw00

array([[    1.,  1660.,  1232., ...,    12.,  7041.,    60.],
       [    1.,  2186.,  1924., ...,    16., 10527.,    56.],
       [    1.,  1428.,  1097., ...,    30.,  8735.,    54.],
       ...,
       [    1.,  2097.,  1915., ...,    20.,  8323.,    49.],
       [    1., 10705.,  2453., ...,    49., 40386.,    99.],
       [    1.,  2989.,  1855., ...,    28.,  4509.,    99.]])

In [59]:
# Construct X matrix
X=raw00[:,(4,0,8,11,16)] # select predictors (note that the first column was removed)
nrow = X.shape[0]
intcpt = np.ones( (nrow,1), ) # create an intercept
X = np.concatenate((intcpt, X), axis=1) # add the intercept to X (i.e X = [intcpt,X] )

In [60]:
# Construct Y vector
Y=raw00[:,15]

* The code above didn't specify whether it is a row or column vector
* Use Y=raw00[:,[15]] to specify it is a column vector

####  <font color='green'> i) Compute LS estimates

$\hat{\beta} = (X^{'}X)^{-1}X^{'}Y$
    
* inv( ) from numpy.linalg
* transpose function
* matrix multiplication

In [61]:
from numpy.linalg import inv
# X transpose and metric multiplication
OLSres = inv(X.T@X)@(X.T@Y)
print(OLSres) # Compare this to the previous result obtained from the statsmodels package

[ 7.94  0.18  4.86  0.   -0.   -0.  ]


####  <font color='green'> ii) Compute the Standard Errors of the Estimates & t-statistics p is regressors

$\hat{\sigma}_{\beta} = Diag \left(\sqrt{\hat{\sigma}^2(X^{'}X)^{-1}} \right)$ where $\hat{\sigma}^2 = \hat{U}^{'}\hat{U}/(n-p-1)$ and $\hat{U} = Y - X\hat{\beta}$
    
$t_{\beta} = | \hat{\beta}/\hat{\sigma}_{\beta} |$
    

In [62]:
# Calculate Residuals
resid = Y-X@OLSres
# Calculate SER (Standard error of the regression)
SER = (resid.T@resid)/(nrow-X.shape[1])
# Calculate SE
SE = np.sqrt(np.diag(SER*inv(X.T@X))) # Compare this to the previous result from the statsmodels package

In [63]:
# Calculate T statistics
Tstat = abs(OLSres/SE)

In [64]:
Tstat

array([5.57, 6.51, 4.97, 6.2 , 4.02, 0.1 ])

### <font color='green'> 2) Least Squares Estimation using Numerical Optimization

$\hat{\beta} = argmin_{\beta} (Y - X\beta)^{'}(Y - X\beta)/n$
    
* Newton Method : https://en.wikipedia.org/wiki/Newton%27s_method_in_optimization
* Optimization in Python: https://scipy-lectures.org/advanced/mathematical_optimization/ & https://realpython.com/python-scipy-cluster-optimize/

In [65]:
from scipy import optimize

In [66]:
# Define loss fn in two ways

# loss function 1
def loss(inpt,Y,X):
    nrow=Y.shape[0]
    loss0=0

    for i in range(0,nrow):
        
            resid = Y[i]-X[i,:]@inpt
            loss0 = loss0+resid*resid
            # can be done simply: loss0+=resid*resid (add and assign)
            
    return loss0

# loss function 2
def loss2(inpt,Y,X):

    resid = Y-X@inpt
    loss0 = resid.T@resid
            
    return loss0

* Optimizer "fmin" : https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fmin.html

In [67]:
# Optimize
## Create a vector of 0 for the start of Beta, dimensioned by the number of variables/columns - column vector
## 0 - number of rows, 1- number of columns
## maxfun and maxiter are stopping functions -
## ftol - if the function almost doesn't change, it will stop
## xtol - if the number is getting close to that value it will stop the smaller the ftol and xtol, the longer it takes
inpt = np.zeros((X.shape[1],1)) # starting value
OLSres2=optimize.fmin(loss2,
                      inpt,
                      args=(Y,X),
                      maxfun=40000,
                      maxiter=40000,
                      ftol=1e-10,
                      xtol=1e-10,
                      disp=True
                     )

Optimization terminated successfully.
         Current function value: 73023.898799
         Iterations: 1486
         Function evaluations: 2368


In [68]:
print(OLSres2)

[ 7.94  0.18  4.86  0.   -0.   -0.  ]


### <font color='darkred'> HW3
* Pick one of your linear regression models in HW2 
* Compute least squares estimates, standard errors of the estimates and t-statistics <ins> using the matrix algebra and optimization algorithm as described above </ins>
* Compare them to the results previously obtained from the statsmodels package

*Matrix Algebra*
This worked well and calculated the coefficients accurately


In [69]:
# Construct X matrix
XH=raw00[:,(0,2,3,4,11,9,17,12,8,14)] # select predictors (note that the first column was removed)
##XH= np.concatenate[XH, XH[:,2]*XH[:,3]]
nrow = XH.shape[0]
intcpt = np.ones( (nrow,1), ) # create an intercept
XH = np.concatenate((intcpt, XH), axis=1) # add the intercept to X (i.e X = [intcpt,X] )

In [75]:
# Construct Y vector
Y=raw00[:,15]

In [76]:
from numpy.linalg import inv
# X transpose and metric multiplication
OLSres = inv(XH.T@XH)@(XH.T@Y)
print(OLSres) # Compare this to the previous result obtained from the statsmodels package

[ 7.43  3.   -0.    0.    0.1  -0.   -0.    0.15  0.06  0.   -0.24]


In [77]:
# Calculate Residuals
resid = Y-XH@OLSres
# Calculate SER (Standard error of the regression)
SER = (resid.T@resid)/(nrow-XH.shape[1])
# Calculate SE
SE = np.sqrt(np.diag(SER*inv(XH.T@XH))) # Compare this to the previous result from the statsmodels package

In [78]:
# Calculate T statistics
Tstat = abs(OLSres/SE)
print(Tstat)

[2.31 2.52 4.45 2.78 3.87 3.39 4.48 6.14 2.25 6.47 2.22]


*Optimization Algorithm* This one did not work as well. This could be due to the number of coefficients I had

In [92]:
# Optimize
inpt = np.zeros((XH.shape[1],1)) # starting value
OLSReg=optimize.fmin(loss2,
                      inpt,
                      args=(Y,XH),
                      ftol=1e-20,
                      xtol=1e-20,
                      disp=True
                     )

In [93]:
print(OLSReg)

[-0.1   0.06 -0.    0.    0.09 -0.   -0.    0.19  0.09  0.   -0.13]


*Previous Regression Result* For comparison, here is the original third model from the previous homework.

In [82]:
# Result from previous regression
import statsmodels.formula.api as smf
import pandas as pd
raw = pd.read_csv('College.csv')

raw.rename(columns = {'perc.alumni':'palum'}, inplace = True)
raw.rename(columns = {'Grad.Rate':'grarate'}, inplace = True)
raw.rename(columns = {'Room.Board':'roomboar'}, inplace = True)
raw.rename(columns = {'F.Undergrad':'fullundergra'}, inplace = True)
raw.rename(columns = {'P.Undergrad':'partundergra'}, inplace = True)
raw.rename(columns = {'S.F.Ratio':'sfrati'}, inplace = True)

raw.head()

,Unnamed: 0,Private,Apps,Accept,Enroll,Top10perc,Top25perc,fullundergra,partundergra,Outstate,roomboar,Books,Personal,PhD,Terminal,sfrati,palum,Expend,grarate
0,Abilene Christian University,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
1,Adelphi University,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
2,Adrian College,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
3,Agnes Scott College,Yes,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
4,Alaska Pacific University,Yes,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15


In [83]:
Reg3 = smf.ols('palum ~ Private + Accept + Enroll + Top10perc + Personal + roomboar + grarate + PhD + Outstate + sfrati', data=raw).fit()
Reg3.params

Intercept         7.426620
Private[T.Yes]    2.999127
Accept           -0.001578
Enroll            0.002756
Top10perc         0.104682
Personal         -0.001831
roomboar         -0.001838
grarate           0.153611
PhD               0.061424
Outstate          0.001013
sfrati           -0.236981
dtype: float64